In [5]:
import pandas as pd
import numpy as np
import datetime
import random
import dash
from dash import dcc, html, Input, Output, State, callback, no_update, callback_context
import dash_bootstrap_components as dbc
import plotly.express as px
import re

print("Generating dummy data...")

COLORS = {
    'background': '#f8f9fa',
    'card': '#ffffff',
    'primary': '#0d47a1', 
    'secondary': '#00897b', 
    'text': '#333333',
    'accent': '#ffa000'
}


# Dummy Data Generation
num_employees_py = 100
departments_global = ['Sales', 'Marketing', 'Engineering', 'HR', 'Finance', 'Operations']
employee_data_global_df = pd.DataFrame({
    'employee_id': [f'EMP{i:03d}' for i in range(1, num_employees_py + 1)],
    'name': [f'Jane Doe' if i == 1 else f'Employee {i}' for i in range(1, num_employees_py + 1)],
    'department': random.choices(departments_global, k=num_employees_py),
    'health_plan': random.choices(['Basic', 'Standard', 'Premium'], weights=[0.3, 0.5, 0.2], k=num_employees_py)
})
employee_data_global_df.set_index('employee_id', inplace=True)

health_plan_details_global = {
    'Basic': {'Clinic Consultations': 2, 'Lab Tests': 3, 'Wellness Program': 1, 'Details': "Basic coverage with ₹50,000 deductible."},
    'Standard': {'Clinic Consultations': 5, 'Lab Tests': 5, 'Wellness Program': 2, 'Details': "Standard coverage with ₹25,000 deductible."},
    'Premium': {'Clinic Consultations': 10, 'Lab Tests': 10, 'Wellness Program': 5, 'Details': "Premium coverage with full access and ₹10,000 deductible."}
}

num_visits_py = 2500
common_issues_global = ['Back Pain', 'Stress-related Conditions', 'Seasonal Allergies', 'Common Cold', 'Headache', 'Digestive Issues']
service_types_global = ['Clinic Visit', 'Video Consult', 'Lab Tests', 'Health Reminders']
visit_data_list_py = []
for _ in range(num_visits_py):
    emp_id = random.choice(employee_data_global_df.index)
    visit_date = datetime.date.today() - datetime.timedelta(days=random.randint(1, 365))
    service = random.choice(service_types_global)
    issue = random.choices(common_issues_global, weights=[0.3, 0.25, 0.1, 0.1, 0.1, 0.15], k=1)[0]
    visit_data_list_py.append({
        'employee_id': emp_id, 'visit_date': visit_date, 'service_type': service,
        'health_issue': issue, 'cost': round(random.uniform(500, 4000), 2)
    })

clinic_visits_df_global = pd.DataFrame(visit_data_list_py).sort_values('visit_date', ascending=False)
clinic_visits_df_global['visit_date'] = pd.to_datetime(clinic_visits_df_global['visit_date'])
clinic_visits_df_global['visit_date_str'] = clinic_visits_df_global['visit_date'].dt.strftime('%Y-%m-%d')
clinic_visits_df_global = pd.merge(clinic_visits_df_global, employee_data_global_df[['department']], left_on='employee_id', right_index=True, how='left')

employee_usage_simulated = {
    emp_id: {
        'Clinic Consultations': random.randint(0, health_plan_details_global[plan]['Clinic Consultations']),
        'Lab Tests': random.randint(0, health_plan_details_global[plan]['Lab Tests']),
        'Wellness Program': random.randint(0, health_plan_details_global[plan]['Wellness Program'])
    } for emp_id, plan in employee_data_global_df['health_plan'].items()
}
print("--- Dummy data generated. ---")

# ---APP INITIALIZATION ---
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP, dbc.icons.FONT_AWESOME], suppress_callback_exceptions=True)
server = app.server

#---HELPER FUNCTION AND COMPONENTS---

def create_navbar():
    return dbc.NavbarSimple(
        children=[
            dbc.NavItem(dbc.NavLink("Employee View", href="/employee")),
            dbc.NavItem(dbc.NavLink("HR View", href="/hr")),
        ],
        brand=html.Div([
            html.I(className="fas fa-heartbeat mr-2"),
            "Aura Health AI"
        ]),
        brand_href="/",
        color=COLORS['primary'],
        dark=True,
        className="shadow-sm"
    )

def create_suggestion_card(title, text, icon, modal_id, is_highlighted=False):
    card_style = {"border": f"2px solid {COLORS['accent']}", "boxShadow": "0 0 .5rem .1rem rgba(255,160,0,.5)"} if is_highlighted else {}
    return dbc.Card([
        dbc.CardBody([
            html.I(className=f"fas {icon} fa-2x mb-3", style={'color': COLORS['primary']}),
            html.H6(title, className="card-title"),
            html.P(text, className="small text-muted"),
            dbc.Button("Learn More", id=f"open-modal-{modal_id}", color="primary", outline=True, size="sm", className="mt-auto")
        ])
    ], className="shadow-sm border-0 h-100 text-center", style=card_style)

def create_info_modal(modal_id, title, icon):
    return dbc.Modal([
        dbc.ModalHeader(html.Div([html.I(className=f"fas {icon} mr-2"), title])),
        dbc.ModalBody([
            html.P(f"Our {title.lower()} offers personalized sessions to help you achieve your wellness goals. "
                   "It includes one-on-one coaching, group workshops, and digital resources accessible anytime. "
                   "Enroll today by asking the AI assistant!"),
            html.H6("Key Benefits:", className="mt-4"),
            dbc.ListGroup([
                dbc.ListGroupItem("✅ Personalized Plans & Guidance"),
                dbc.ListGroupItem("✅ Flexible Scheduling Options"),
                dbc.ListGroupItem("✅ Expert-led Sessions & Materials"),
            ], flush=True),
        ]),
        dbc.ModalFooter(dbc.Button("Close", id=f"close-modal-{modal_id}", className="ml-auto"))
    ], id=f"modal-{modal_id}", centered=True)

def create_roi_table():
    wellness_data = [
        {'Program': 'Mindfulness Workshops', 'Participation Rate': 0.45, 'Expected Health Impact': 'Reduced stress', 'Simulated Cost Reduction Per Participant': 1500},
        {'Program': 'Corporate Yoga', 'Participation Rate': 0.30, 'Expected Health Impact': 'Improved musculoskeletal health', 'Simulated Cost Reduction Per Participant': 1000},
        {'Program': 'Annual Health Checks', 'Participation Rate': 0.85, 'Expected Health Impact': 'Early detection', 'Simulated Cost Reduction Per Participant': 2500},
        {'Program': 'Nutrition Counseling', 'Participation Rate': 0.20, 'Expected Health Impact': 'Better diet habits', 'Simulated Cost Reduction Per Participant': 800},
    ]

    total_savings = 0
    table_rows = []
    for program in wellness_data:
        savings = program['Participation Rate'] * num_employees_py * program['Simulated Cost Reduction Per Participant']
        total_savings += savings
        table_rows.append(html.Tr([
            html.Td(program['Program']),
            html.Td(f"{program['Participation Rate']:.1%}"),
            html.Td(program['Expected Health Impact']),
            html.Td(f"₹{program['Simulated Cost Reduction Per Participant']:,.0f}"),
            html.Td(f"₹{savings:,.1f}")
        ]))
    
    # Add the total row
    total_row = html.Tr([
        html.Td(html.B("Total Simulated ROI across programs:"), colSpan=4, className="text-right font-weight-bold"),
        html.Td(html.B(f"₹{total_savings:,.1f}"), className="font-weight-bold")
    ], style={'backgroundColor': 'rgba(0,0,0,0.05)'})
    table_rows.append(total_row)

    table_header = [html.Thead(html.Tr([
        html.Th("Program"),
        html.Th("Participation Rate"),
        html.Th("Expected Health Impact"),
        html.Th("Simulated Cost Reduction Per Participant"),
        html.Th("Simulated Total Savings INR")
    ]))]
    
    table_body = [html.Tbody(table_rows)]
    
    return dbc.Table(table_header + table_body, bordered=True, striped=True, hover=True, responsive=True)

def get_landing_page_layout():
    return html.Div([
        create_navbar(),
        dbc.Container([
            dbc.Row(dbc.Col(html.H1("Welcome to the Future of Employee Wellness", className="text-center my-5", style={'color': COLORS['primary']}), width=12)),
            dbc.Row([
                dbc.Col(dbc.Card([
                    dbc.CardBody([
                        html.I(className="fas fa-user-tie fa-4x my-4 text-primary"),
                        html.H4("Employee Portal", className="card-title"),
                        html.P("Access your benefits, book appointments, and get personalized wellness advice.", className="card-text"),
                        dbc.Button("Enter Your Dashboard", href="/employee", color="primary", className="mt-auto")
                    ])
                ], className="text-center shadow border-0 h-100"), md=5),
                dbc.Col(dbc.Card([
                    dbc.CardBody([
                        html.I(className="fas fa-chart-line fa-4x my-4 text-secondary"),
                        html.H4("HR Analytics Portal", className="card-title"),
                        html.P("Unlock workforce health trends, analyze ROI, and deploy targeted wellness programs.", className="card-text"),
                        dbc.Button("Enter HR Dashboard", href="/hr", color="secondary", className="mt-auto")
                    ])
                ], className="text-center shadow border-0 h-100"), md=5),
            ], justify="center", className="mt-4")
        ], fluid=False)
    ], style={'backgroundColor': COLORS['background'], 'minHeight': '100vh'})

def get_employee_page_layout():
    return html.Div([
        create_navbar(),
        dbc.Container([
            dcc.Store(id='employee-chat-history-store', data=[]),
            dcc.Store(id='employee-convo-state-store', data={'awaiting': None, 'booking_details': {}}),
            dbc.Row([
                dbc.Col(html.H2("My Health Dashboard", className="font-weight-bold"), width="auto"),
                dbc.Col(dcc.Dropdown(
                    id='employee-id-selector',
                    options=[{'label': f"{v['name']} ({k})", 'value': k} for k, v in employee_data_global_df.to_dict('index').items()],
                    value='EMP001', clearable=False
                ), width=4)
            ], align="center", className="my-4"),
            dbc.Row([
                dbc.Col([
                    dbc.Card(id='employee-welcome-card', className="mb-4 p-3 shadow-sm border-0"),
                    dbc.Card([
                        dbc.CardHeader(html.H5("Your Health Benefits Usage")),
                        dbc.CardBody(id='employee-benefits-progress')
                    ], className="mb-4 shadow-sm border-0"),
                    dbc.Card([
                        dbc.CardHeader(html.H5("Recent Activity")),
                        dbc.CardBody(id='employee-recent-activity', style={'maxHeight': '250px', 'overflowY': 'auto'})
                    ], className="mb-4 shadow-sm border-0"),
                ], md=5),
                dbc.Col([
                    dbc.Card([
                        dbc.CardHeader(html.H5("AI Assistant")),
                        dbc.CardBody([
                            html.Div(id='employee-chat-window', className="mb-3 p-2", style={'height': '200px', 'overflowY': 'auto', 'border': '1px solid #ddd', 'borderRadius': '5px'}),
                            dbc.InputGroup([
                                dbc.Input(id='employee-chat-input', placeholder="Ask to 'book a visit'...", n_submit=0),
                                dbc.Button(html.I(className="fas fa-paper-plane"), id='employee-send-button', n_clicks=0, color="primary")
                            ])
                        ])
                    ], className="mb-4 shadow-sm border-0"),
                    dbc.Card([
                        dbc.CardHeader(html.H5("Personalized Suggestions")),
                        dbc.CardBody(id="employee-dynamic-suggestions")
                    ], className="shadow-sm border-0")
                ], md=7)
            ])
        ], fluid=True, className="py-4"),
        create_info_modal("mindfulness", "Mindfulness Program", "fa-brain"),
        create_info_modal("nutrition", "Nutrition Coaching", "fa-carrot"),
        create_info_modal("fitness", "Fitness Challenges", "fa-dumbbell"),
        create_info_modal("telehealth", "Telehealth Consults", "fa-laptop-medical"),
    ], style={'backgroundColor': COLORS['background'], 'minHeight': '100vh'})

def get_hr_page_layout():
    return html.Div([
        create_navbar(),
        dbc.Container([
            dcc.Store(id='hr-chat-history-store', data=[]),
            dcc.Download(id="download-csv"),
            html.H2("HR Analytics Dashboard", className="font-weight-bold my-4"),
            
            dbc.Row([
                dbc.Col(dbc.Card(dbc.CardBody([
                    html.H5("Filters & Reporting", className="card-title"),
                    dcc.Dropdown(
                        id='hr-department-filter',
                        options=[{'label': 'All Departments', 'value': 'All'}] + [{'label': dept, 'value': dept} for dept in departments_global],
                        value='All', placeholder="Filter by Department", className="mb-3"
                    ),
                    dbc.Button([html.I(className="fas fa-download mr-2"), "Generate CSV Report"], id="btn-download-csv", color="success", className="w-100")
                ])), md=8),
                dbc.Col(dbc.Card([
                    dbc.CardHeader("💡 AI-Powered Insights"),
                    dbc.CardBody(id='hr-ai-insights-card', style={'minHeight': '140px'})
                ]), md=4),
            ], className="mb-4"),

            dbc.Tabs([
                dbc.Tab(label="Dashboard Overview", children=[
                    dbc.Card(className="mt-4", children=[
                        dbc.CardHeader(html.H5("Key Performance Indicators")),
                        dbc.CardBody(id='hr-kpi-cards')
                    ]),
                    dbc.Row([
                        dbc.Col(dbc.Card([
                            dbc.CardHeader(html.H5("📈 Monthly Clinic Visits Trend")),
                            dbc.CardBody(dcc.Graph(id='hr-monthly-trend-graph', config={'displayModeBar': False}))
                        ], className="mt-4"), md=12),
                    ])
                ]),
                dbc.Tab(label="Health Issue Analysis", children=[
                    dbc.Card([
                        dbc.CardHeader(html.H5("📊 Health Issues by Department")),
                        dbc.CardBody(dcc.Graph(id='hr-dept-issue-graph', config={'displayModeBar': False}))
                    ], className="mt-4")
                ]),
                dbc.Tab(label="Cost Analysis", children=[
                    dbc.Card(
                        dbc.CardBody(dcc.Graph(id='hr-cost-analysis-graph', config={'displayModeBar': False})),
                        className="mt-4"
                    ),
                    dbc.Card([
                        dbc.CardHeader(html.H5("Simulated ROI for Wellness Programs")),
                        dbc.CardBody(create_roi_table()) # <-- New table added here
                    ], className="mt-4")
                ]),
                dbc.Tab(label="Employee Lookup", children=[
                    dbc.Card(className="mt-4", body=True, children=[
                        html.H5("Search for an Employee"),
                        dcc.Dropdown(
                            id='hr-employee-lookup-dropdown',
                            options=[{'label': f"{v['name']} ({k})", 'value': k} for k, v in employee_data_global_df.to_dict('index').items()],
                            placeholder="Select an employee to view their details..."
                        ),
                        html.Div(id='hr-employee-lookup-output', className="mt-4")
                    ])
                ]),
                dbc.Tab(label="Chat with AI", children=[
                    dbc.Card(className="mt-4", children=[
                        dbc.CardHeader(html.H5("Chat with Health AI Assistant")),
                        dbc.CardBody([
                            html.Div(id='hr-chat-window', className="mb-3 p-2", style={'height': '400px', 'overflowY': 'auto', 'border': '1px solid #ddd', 'borderRadius': '5px'}),
                            dbc.InputGroup([
                                dbc.Input(id='hr-chat-input', placeholder="Ask 'what's our ROI?'...", n_submit=0),
                                dbc.Button(html.I(className="fas fa-paper-plane"), id='hr-send-button', n_clicks=0, color="primary")
                            ])
                        ])
                    ])
                ])
            ])
            
        ], fluid=True, className="py-4"),
    ], style={'backgroundColor': COLORS['background'], 'minHeight': '100vh'})

app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div(id='page-content')
])

@callback(Output('page-content', 'children'), Input('url', 'pathname'))
def display_page(pathname):
    if pathname == '/employee': return get_employee_page_layout()
    elif pathname == '/hr': return get_hr_page_layout()
    else: return get_landing_page_layout()


# --- CHATBOT LOGIC --- (same as before)
def get_employee_aura_response(prompt: str, emp_id: str, state: dict):
    prompt_lower = prompt.lower().strip()
    response = "I'm sorry, I don't understand. I can help with booking appointments, checking benefits, setting reminders, and suggesting doctors."
    next_state = state.copy()

    if next_state.get('awaiting') == 'booking_confirm':
        details = next_state['booking_details']
        if 'yes' in prompt_lower:
            response = f"✅ Great! I've simulated booking a **{details['service']}** for you. A confirmation will be sent shortly."
        else:
            response = " booking has been cancelled. Is there anything else?"
        next_state = {'awaiting': None, 'booking_details': {}}
        return response, next_state

    if any(k in prompt_lower for k in ['book', 'visit', 'appointment', 'schedule']):
        service = next((s for s in service_types_global if s.lower() in prompt_lower), None)
        if service:
            response = f"Sure, I can help book a **{service}**. To confirm, should I proceed? (Type 'yes' or 'no')"
            next_state = {'awaiting': 'booking_confirm', 'booking_details': {'service': service}}
        else:
            response = f"Which service would you like? Choose from: **{', '.join(service_types_global)}**."
    elif any(k in prompt_lower for k in ['benefit', 'plan', 'covered']):
        plan_name = employee_data_global_df.loc[emp_id, 'health_plan']
        plan_details = health_plan_details_global.get(plan_name)
        response = (f"You are on the **{plan_name} Plan**. Here are your key annual limits:\n\n"
                    f"- **Clinic Consultations:** {plan_details['Clinic Consultations']} per year.\n"
                    f"- **Lab Tests:** {plan_details['Lab Tests']} per year.\n"
                    f"- **Wellness Programs:** {plan_details['Wellness Program']} covered.\n\n"
                    f"*{plan_details['Details']}*")
    elif any(k in prompt_lower for k in ['doctor', 'specialist', 'see for']):
        if 'back pain' in prompt_lower:
            response = "For back pain, people often consult a General Practitioner first, who may refer them to a Physiotherapist. Our on-site clinic can help."
        elif 'stress' in prompt_lower or 'anxiety' in prompt_lower:
            response = "For stress-related concerns, a consultation with a General Practitioner or a mental health professional is a great first step. We have resources available."
        else:
            response = "I can provide general guidance on specialists. What symptoms are you experiencing?"
    elif any(k in prompt_lower for k in ['remind', 'reminder']):
        response = "I can help with that! What would you like a reminder for, and when? (This is a simulation)"
        next_state['awaiting'] = 'reminder_details'
    elif any(k in prompt_lower for k in ['hello', 'hi', 'hey']):
        response = "Hello! I'm Aura. How can I assist you? You can ask me to 'book a clinic visit' or 'check my plan'."
    return response, next_state

def get_hr_aura_response(prompt: str):
    prompt_lower = prompt.lower().strip()
    df = clinic_visits_df_global
    response = "I can answer questions about aggregate trends, ROI, and department-level data. Try 'what are the top issues?'"

    if any(k in prompt_lower for k in ['roi', 'return on investment']):
        total_cost = df['cost'].sum()
        preventative_savings = len(df[df['service_type'] == 'Health Reminders']) * 1500
        roi = (preventative_savings / total_cost) * 100 if total_cost > 0 else 0
        response = (f"📈 Based on simulated data, the current wellness program **ROI is approximately {roi:.2f}%**. "
                    f"This is driven by preventative care reminders reducing potential future costs.")
    elif any(k in prompt_lower for k in ['top issue', 'common condition']):
        top_issues = df['health_issue'].value_counts().nlargest(3)
        response = "The **top 3 recurring health issues** are:\n"
        for i, (issue, count) in enumerate(top_issues.items()):
            response += f"\n{i+1}. **{issue}:** {count} cases."
    elif any(k in prompt_lower for k in ['department usage', 'department visit']):
        dept_usage = df['department'].value_counts().nlargest(3)
        response = "The **departments with the highest clinic utilization** are:\n"
        for i, (dept, count) in enumerate(dept_usage.items()):
            response += f"\n{i+1}. **{dept}:** {count} visits."
    elif any(k in prompt_lower for k in ['hello', 'hi', 'hey']):
        response = "Hello! As the HR Health AI, I can provide aggregate insights. Try asking 'what's our benefits ROI?'"
    return response

# --- Employee Page Callbacks ---
app.clientside_callback(
    """
    function(n1, n2) {
        if (n1 > 0 || n2 > 0) {
            const ctx = dash_clientside.callback_context;
            const trigger_id = ctx.triggered[0].prop_id.split('.')[0];
            if (trigger_id === 'quick-action-book') {
                return 'Book a Clinic Visit';
            } else if (trigger_id === 'quick-action-plan') {
                return 'What is covered in my plan?';
            }
        }
        return dash_clientside.no_update;
    }
    """,
    Output('employee-chat-input', 'value', allow_duplicate=True),
    Input('quick-action-book', 'n_clicks'),
    Input('quick-action-plan', 'n_clicks'),
    prevent_initial_call=True
)

@callback(
    [Output('employee-welcome-card', 'children'),
     Output('employee-benefits-progress', 'children'),
     Output('employee-recent-activity', 'children'),
     Output('employee-dynamic-suggestions', 'children')],
    Input('employee-id-selector', 'value')
)
def update_employee_dashboard(emp_id):
    if not emp_id: return no_update, no_update, no_update, no_update
    emp_info = employee_data_global_df.loc[emp_id]
    plan_name = emp_info['health_plan']
    plan_limits = health_plan_details_global[plan_name]
    usage = employee_usage_simulated[emp_id]

    welcome_card = [
        html.H4(f"Welcome, {emp_info['name']}!", className="font-weight-bold", style={'color': COLORS['primary']}),
        html.P("Here are your quick actions and health summary.", className="text-muted small"),
        html.Div([
            dbc.Button("Book a Visit", id="quick-action-book", size="sm", className="mr-2"),
            dbc.Button("Check My Plan", id="quick-action-plan", size="sm", color="secondary")
        ], className="mt-2")
    ]
    benefits_progress = []
    for benefit, limit in plan_limits.items():
        if isinstance(limit, int):
            used = usage.get(benefit, 0)
            percentage = (used / limit) * 100 if limit > 0 else 0
            benefits_progress.append(html.Div([
                html.P(f"{benefit}: {used} of {limit} used", className="mb-1 font-weight-bold small"),
                dbc.Progress(value=percentage, style={'height': '10px'}, color=COLORS['secondary'])
            ], className="mb-3"))

    emp_visits = clinic_visits_df_global[clinic_visits_df_global['employee_id'] == emp_id]
    
    if emp_visits.empty:
        activity_feed = [dbc.ListGroupItem("No recent activity found.")]
    else:
        activity_feed = [
            dbc.ListGroupItem([
                html.Div([
                    html.B(f"{row['service_type']}"),
                    html.Span(f" on {row['visit_date_str']}", className="small text-muted float-right")
                ]),
                html.Div(f"Reason: {row['health_issue']}", className="small")
            ]) for index, row in emp_visits.head(5).iterrows()
        ]

    highlight = None
    if not emp_visits.empty:
        top_issue = emp_visits['health_issue'].mode()
        if not top_issue.empty:
            top_issue = top_issue[0]
            if top_issue == 'Back Pain':
                highlight = 'fitness'
            elif top_issue == 'Stress-related Conditions':
                highlight = 'mindfulness'

    suggestions = dbc.Row([
        dbc.Col(create_suggestion_card("Mindfulness", "Reduce stress & improve focus.", "fa-brain", "mindfulness", is_highlighted=(highlight=='mindfulness')), md=6, className="mb-3"),
        dbc.Col(create_suggestion_card("Nutrition", "Get personalized diet plans.", "fa-carrot", "nutrition", is_highlighted=(highlight=='nutrition')), md=6, className="mb-3"),
        dbc.Col(create_suggestion_card("Fitness", "Join challenges & stay active.", "fa-dumbbell", "fitness", is_highlighted=(highlight=='fitness')), md=6, className="mb-3"),
        dbc.Col(create_suggestion_card("Telehealth", "Consult doctors from home.", "fa-laptop-medical", "telehealth", is_highlighted=(highlight=='telehealth')), md=6, className="mb-3"),
    ])

    return welcome_card, benefits_progress, dbc.ListGroup(activity_feed, flush=True), suggestions

@callback(
    [Output('employee-chat-window', 'children'),
     Output('employee-chat-input', 'value', allow_duplicate=True),
     Output('employee-chat-history-store', 'data'),
     Output('employee-convo-state-store', 'data')],
    [Input('employee-send-button', 'n_clicks'), Input('employee-chat-input', 'n_submit')],
    [State('employee-chat-input', 'value'),
     State('employee-chat-history-store', 'data'),
     State('employee-convo-state-store', 'data'),
     State('employee-id-selector', 'value')],
    prevent_initial_call=True
)
def handle_employee_chat(send_clicks, submit, user_message, chat_history, convo_state, emp_id):
    if not user_message or not user_message.strip(): return no_update
    if not chat_history: chat_history = []
    chat_history.append({'sender': 'You', 'message': user_message})
    aura_response_text, updated_convo_state = get_employee_aura_response(user_message, emp_id, convo_state)
    chat_history.append({'sender': 'Aura AI', 'message': aura_response_text})
    chat_bubbles = [dbc.Row(dbc.Col(dbc.Alert(dcc.Markdown(msg['message']), color="primary" if msg['sender'] == 'You' else "light", style={'width': 'fit-content', 'maxWidth': '80%'}), width="auto"), justify="end" if msg['sender'] == 'You' else "start", className="mb-2") for msg in chat_history]
    return chat_bubbles, "", chat_history, updated_convo_state

@callback(
    [Output(f"modal-{modal_id}", "is_open") for modal_id in ["mindfulness", "nutrition", "fitness", "telehealth"]],
    [Input(f"open-modal-{modal_id}", "n_clicks") for modal_id in ["mindfulness", "nutrition", "fitness", "telehealth"]] +
    [Input(f"close-modal-{modal_id}", "n_clicks") for modal_id in ["mindfulness", "nutrition", "fitness", "telehealth"]],
    [State(f"modal-{modal_id}", "is_open") for modal_id in ["mindfulness", "nutrition", "fitness", "telehealth"]],
    prevent_initial_call=True
)
def toggle_modals(*args):
    ctx = callback_context
    if not ctx.triggered: return [False] * 4
    
    trigger_id = ctx.triggered[0]["prop_id"].split(".")[0]
    modal_name = trigger_id.split("-")[-1]
    
    is_open_states = {name: args[i+8] for i, name in enumerate(["mindfulness", "nutrition", "fitness", "telehealth"])}

    if modal_name in is_open_states:
        is_open_states[modal_name] = not is_open_states[modal_name]

    return list(is_open_states.values())


# --- HR Page Callbacks ---
@callback(
    [Output('hr-ai-insights-card', 'children'),
     Output('hr-kpi-cards', 'children'),
     Output('hr-dept-issue-graph', 'figure'),
     Output('hr-monthly-trend-graph', 'figure'),
     Output('hr-cost-analysis-graph', 'figure')],
    Input('hr-department-filter', 'value')
)
def update_hr_dashboard(department):
    df = clinic_visits_df_global
    if department and department != 'All':
        df = df[df['department'] == department]

    if df.empty:
        no_data_fig = {'layout': {'xaxis': {'visible': False}, 'yaxis': {'visible': False}, 'annotations': [{'text': 'No data for this selection', 'showarrow': False}]}}
        return html.P("No data available for this filter."), [], no_data_fig, no_data_fig, no_data_fig

    top_issue = df['health_issue'].value_counts().idxmax()
    issue_percent = (df['health_issue'].value_counts().max() / len(df)) * 100
    ai_insight_content = [
        html.P(["Analysis shows ", html.B(f"{top_issue}", style={'color': COLORS['primary']}), f" is the leading issue, affecting {issue_percent:.1f}% of cases."]),
        html.P("Consider targeted workshops on ergonomics or stress management.", className="small text-muted")
    ]
    
    # KPIs
    total_visits = len(df)
    total_cost = df['cost'].sum()
    kpi_cards = dbc.Row([
        dbc.Col(dbc.Card(dbc.CardBody([html.P("Total Visits", className="small text-muted mb-1"), html.H4(f"{total_visits:,}", className="font-weight-bold")]))),
        dbc.Col(dbc.Card(dbc.CardBody([html.P("Total Costs", className="small text-muted mb-1"), html.H4(f"₹{total_cost:,.0f}")]))),
        dbc.Col(dbc.Card(dbc.CardBody([html.P("Avg Cost/Visit", className="small text-muted mb-1"), html.H4(f"₹{(total_cost/total_visits):,.2f}")]))),
        dbc.Col(dbc.Card(dbc.CardBody([html.P("Top Issue", className="small text-muted mb-1"), html.H4(top_issue, style={'fontSize': '1.1rem'})]))),
    ])

    # Department Issue Graph
    dept_issue_counts = df.groupby(['department', 'health_issue']).size().reset_index(name='count')
    fig_dept = px.bar(dept_issue_counts, x='department', y='count', color='health_issue', color_discrete_sequence=px.colors.qualitative.Pastel)
    fig_dept.update_layout(margin=dict(l=20, r=20, t=10, b=20), paper_bgcolor=COLORS['card'], plot_bgcolor=COLORS['card'], legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))
    
    # Monthly Trend Graph
    monthly_visits = df.resample('M', on='visit_date').size().reset_index(name='count')
    monthly_visits['visit_date'] = monthly_visits['visit_date'].dt.strftime('%b %Y')
    fig_trend = px.area(monthly_visits, x='visit_date', y='count', markers=True, color_discrete_sequence=[COLORS['secondary']])
    fig_trend.update_layout(margin=dict(l=20, r=20, t=10, b=20), paper_bgcolor=COLORS['card'], plot_bgcolor=COLORS['card'])
    fig_trend.update_xaxes(title_text=None)
    fig_trend.update_yaxes(title_text='Visits')

    # Cost Analysis Graph
    cost_by_service = df.groupby('service_type')['cost'].sum().reset_index()
    fig_cost = px.pie(cost_by_service, names='service_type', values='cost', title="Cost Distribution by Service Type", hole=0.4, color_discrete_sequence=px.colors.sequential.Teal)
    fig_cost.update_layout(margin=dict(l=20, r=20, t=40, b=20), paper_bgcolor=COLORS['card'])

    return ai_insight_content, kpi_cards, fig_dept, fig_trend, fig_cost

@callback(
    Output("download-csv", "data"),
    Input("btn-download-csv", "n_clicks"),
    State("hr-department-filter", "value"),
    prevent_initial_call=True
)
def download_hr_report(n_clicks, department):
    df = clinic_visits_df_global
    if department and department != 'All':
        df = df[df['department'] == department]
    return dcc.send_data_frame(df.to_csv, f"aura_health_report_{department.lower()}_{datetime.date.today()}.csv", index=False)

@callback(
    Output('hr-employee-lookup-output', 'children'),
    Input('hr-employee-lookup-dropdown', 'value'),
    prevent_initial_call=True
)
def update_employee_lookup(emp_id):
    if not emp_id:
        return dbc.Alert("Please select an employee to view their details.", color="info")

    emp_info = employee_data_global_df.loc[emp_id]
    emp_visits = clinic_visits_df_global[clinic_visits_df_global['employee_id'] == emp_id]

    visit_history = [
        dbc.ListGroupItem([
            html.Div([
                html.B(f"{row['service_type']} for {row['health_issue']}"),
                html.Span(f"₹{row['cost']:.2f}", className="float-right")
            ]),
            html.Div(f"Date: {row['visit_date_str']}", className="small text-muted")
        ]) for _, row in emp_visits.iterrows()
    ]
    if not visit_history:
        visit_history = [dbc.ListGroupItem("No visit history found for this employee.")]

    return html.Div([
        dbc.Alert([
            html.I(className="fas fa-exclamation-triangle mr-2"),
            "You are viewing sensitive employee health information. Please handle with care."
        ], color="warning"),
        dbc.Row([
            dbc.Col(dbc.Card(dbc.CardBody([
                html.H5(emp_info['name'], className="card-title"),
                html.P(f"Department: {emp_info['department']}"),
                html.P(f"Health Plan: {emp_info['health_plan']}")
            ])), width=4),
            dbc.Col(dbc.Card([
                dbc.CardHeader("Visit History"),
                dbc.ListGroup(visit_history, flush=True, style={'maxHeight': '400px', 'overflowY': 'auto'})
            ]), width=8)
        ])
    ])

@callback(
    [Output('hr-chat-window', 'children'),
     Output('hr-chat-input', 'value'),
     Output('hr-chat-history-store', 'data')],
    [Input('hr-send-button', 'n_clicks'), Input('hr-chat-input', 'n_submit')],
    [State('hr-chat-input', 'value'), State('hr-chat-history-store', 'data')],
    prevent_initial_call=True
)
def handle_hr_chat(n_clicks, n_submit, user_message, chat_history):
    if not user_message or not user_message.strip(): return no_update
    if not chat_history: chat_history = []
    chat_history.append({'sender': 'You', 'message': user_message})
    aura_response_text = get_hr_aura_response(user_message)
    chat_history.append({'sender': 'Aura AI', 'message': aura_response_text})
    chat_bubbles = [dbc.Row(dbc.Col(dbc.Alert(dcc.Markdown(msg['message']), color="primary" if msg['sender'] == 'You' else "light", style={'width': 'fit-content', 'maxWidth': '80%'}), width="auto"), justify="end" if msg['sender'] == 'You' else "start", className="mb-2") for msg in chat_history]
    return chat_bubbles, "", chat_history

if __name__ == '__main__':
    app.run(debug=True, port=8052)

Generating dummy data...
--- Dummy data generated. ---
